In [1]:
# Importing needed library
import pandas as pd
import numpy as np
import beta_glu
import pandascharm as pc
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo
from scipy.fft import fft, ifft

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from pyaaisc import Aaindex
from Bio import AlignIO


/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
df = pd.read_excel('../beta-glucosidase/data/KcatKm w_ Temperatures.xlsx')

In [3]:
output = 'pNP-Glc kcat/Km (1/smM)'
encoding = 'trigram'
aln = '../beta-glucosidase/data/kcatKm w_Temperatures.aln'

In [4]:
X,y,holder = beta_glu.encode_temp(encoding, output, df, aln, key = None)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=101)

In [6]:
all_result = []

In [7]:
# DT Tuning
def DT(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid={ "max_features": [2,4,6,8,10,12], 'random_state':[20]}
    tree_reg=DecisionTreeRegressor()
    if jack == False:
        grid_search=GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(tree_reg, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    DT_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return DT_pred, pred_training, param

In [8]:
DT_pred , DT_train, param = DT(X_train, y_train, X_val)

In [9]:
all_result.append([output, "Decision Tree Regression", encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_val, DT_pred)), r2_score(y_train, DT_train),r2_score(y_val, DT_pred), str(param)]) 

In [10]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'trigram',
  nan,
  2.3099881679993453,
  480.08191913170185,
  0.9999989036035922,
  0.9635297519349539,
  "{'max_features': 2, 'random_state': 20}"]]

In [11]:
# SVM Tuning
def SVM(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid = {'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5]}
 
    model = SVR()
    if jack == False:
        grid_search=GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(model, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    SVM_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return SVM_pred, pred_training, param

In [12]:
SVM_pred , SVM_train, param = SVM(X_train, y_train, X_val)

In [13]:
all_result.append([output, "Support Vector Machine Regression", encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_val, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_val, SVM_pred), param])

In [14]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'trigram',
  nan,
  2.3099881679993453,
  480.08191913170185,
  0.9999989036035922,
  0.9635297519349539,
  "{'max_features': 2, 'random_state': 20}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Support Vector Machine Regression',
  'trigram',
  nan,
  2255.53322487129,
  2550.8287879803693,
  -0.04531347957926091,
  -0.029604985844223997,
  {'C': 10, 'coef0': 0.01, 'degree': 3}]]

In [15]:
param

{'C': 10, 'coef0': 0.01, 'degree': 3}

In [16]:
# Random Forest Tuning
def RF(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 50, stop = 150, num = 10)]
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Create the random grid
    param_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}
    rf = RandomForestRegressor()
    if jack == False:
        grid_search=GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(rf, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    RF_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return RF_pred, pred_training, param

In [17]:
RF_pred , RF_train, param = RF(X_train, y_train, X_val)

In [18]:
all_result.append([output, "Random Forest Regression", encoding,holder,np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_val, RF_pred)), r2_score(y_train, RF_train),r2_score(y_val, RF_pred),str(param)])

In [19]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'trigram',
  nan,
  2.3099881679993453,
  480.08191913170185,
  0.9999989036035922,
  0.9635297519349539,
  "{'max_features': 2, 'random_state': 20}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Support Vector Machine Regression',
  'trigram',
  nan,
  2255.53322487129,
  2550.8287879803693,
  -0.04531347957926091,
  -0.029604985844223997,
  {'C': 10, 'coef0': 0.01, 'degree': 3}],
 ['pNP-Glc kcat/Km (1/smM)',
  'Random Forest Regression',
  'trigram',
  nan,
  275.20675267034426,
  553.1241509986928,
  0.9844379694027354,
  0.9515879737071933,
  "{'max_depth': 70, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}"]]

In [20]:
param

{'max_depth': 70,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

In [21]:
# XGBR tuning
def XGBR(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
    
    if jack == False:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    XGB_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return XGB_pred, pred_training, param


In [ ]:
XG_pred, XG_train, param = XGBR(X_train, y_train, X_val)

In [ ]:
all_result.append([output,"XGB Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, XG_train)), np.sqrt(metrics.mean_squared_error(y_val, XG_pred)), r2_score(y_train, XG_train),r2_score(y_val, XG_pred),str(param)]) 

In [ ]:
all_result

In [ ]:
param

In [ ]:
# NNR Tuning
def NNR(X_train, y_train, X_val, jack = False):
    
    knife = len(X_train)-1
    param_grid = {"hidden_layer_sizes": [1,50], "activation": ["identity", "logistic", "tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005]}
    regr = MLPRegressor(random_state=101, max_iter=100).fit(X_train, y_train)
    if jack == False:
        grid_search=GridSearchCV(regr, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(regr, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    NNR_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return NNR_pred, pred_training, param

In [ ]:
NNR_pred, NNR_train, param = NNR(X_train, y_train, X_val)

In [ ]:
all_result.append([output,"Neural Network Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_val, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_val, NNR_pred),str(param)]) 

In [ ]:
all_result

In [ ]:
#Elastic Net

def EN(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid = {"max_iter": [1, 5, 10], "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],"l1_ratio": np.arange(0.0, 1.0, 0.1)}
    ENmodel = ElasticNet()
    if jack == False:
        grid_search=GridSearchCV(ENmodel, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(ENmodel, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    EN_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return EN_pred, pred_training,param

In [ ]:
EN_pred , EN_train, param = EN(X_train, y_train, X_val)

In [ ]:
all_result.append([output,"Elastic Net Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_val, EN_pred)), r2_score(y_train, EN_train),r2_score(y_val, EN_pred),str(param)])

In [ ]:
all_result

In [ ]:
dfResults = pd.DataFrame(all_result, columns=['Output' , 'Algorithm','Encoding Method' ,'Code', "RMSE Training", 'RMSE Val',"R^2 train","R^2 Val","Best_Parameter"])

In [ ]:
dfResults

In [ ]:
name = encoding + "_hyperparameter_tune_new.xlsx"
dfResults.to_excel(name)  